# Census Data Cleaning
Keanna Knebel

---

In [349]:
# import packages
import pandas as pd
import re
import os

In [350]:
# Options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [351]:
census_2001 = "../../data/raw/census_2001.csv"
census_2006 = "../../data/raw/census_2006.csv"
census_2011 = "../../data/raw/census_2011.csv"
census_2016 = "../../data/raw/census_2016.csv"

In [352]:
def clean_census(file, year):
    
    column_names= ['Variable', 'Arbutus-Ridge', 'Downtown', 'Dunbar-Southlands',
               'Fairview', 'Grandview-Woodland', 'Hastings-Sunrise',
               'Kensington-Cedar Cottage', 'Kerrisdale', 'Killarney', 'Kitsilano',
               'Marpole', 'Mount Pleasant', 'Oakridge', 'Renfrew-Collingwood',
               'Riley Park', 'Shaughnessy', 'South Cambie', 'Strathcona',
               'Sunset', 'Victoria-Fraserview', 'West End', 'West Point Grey',
               'Vancouver CSD', 'Vancouver CMA']
   
    # read in csv file as dataframe
    df = pd.read_csv(file, encoding='latin-1', skiprows=4)
    
    # remove 'ID' column if present
    df.drop(columns='ID', inplace=True, errors='ignore')
    
    # rename columns 
    df.set_axis(column_names, axis=1, inplace=True)
    
    # remove empty rows
    df.dropna(0, 'all', inplace=True)
    
    # remove leading whitespace from variables
    df.Variable = df.Variable.apply(lambda x: (x.lstrip()).rstrip())
    df.drop(df[df.Variable.str.contains('20%.*data', flags=re.IGNORECASE) ].index , inplace=True)
    
    # convert all data to strings except NaN values
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: str(x) if x == x else x)
    
    # convert data to float
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: re.sub("[-]", "0", x) if x == x else x)
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: float(re.sub("[,$]", "", x)) if x == x else x)
    
    # Create the census subdirectory for given year if it doesn't exist
    os.makedirs('../../data/processed/census_' + str(year), exist_ok=True) 
    
    # divide the census datasets into subgroups
    sub_dict = create_subgroup_dict(df, year)
    
    # clean the datatables by topics
    census_dict = clean_age(sub_dict, year)
    census_dict = clean_marital_status(census_dict, year)
    census_dict = clean_private_households(census_dict, year)
    census_dict = clean_couple_fam_structure(census_dict, year)
    census_dict = clean_language_detailed(census_dict, year)
    census_dict = clean_official_language(census_dict, year)
    census_dict = clean_structural_dwelling_type(census_dict, year)
    census_dict = clean_household_size(census_dict, year)
    census_dict = clean_lone_parent(census_dict, year)
    census_dict = clean_immigration_age(census_dict, year)
    census_dict = clean_immigration_period(census_dict, year)
    census_dict = clean_birth_place(census_dict, year)
    census_dict = clean_shelter_tenure(census_dict, year)
    census_dict = clean_visible_minority(census_dict, year)
    
    #### INCOMPLETE HELPER FUNCTIONS ####
    #census_dict = clean_household_type(census_dict, year)
    #census_dict = clean_census_family_children(census_dict, year)
    #census_dict = clean_aboriginal(census_dict, year)
    #census_dict = clean_citizenship(census_dict, year)
    #census_dict = clean_worker_class(census_dict, year)
    #census_dict = clean_education(census_dict, year)
    #census_dict = clean_ethnic_origin(census_dict, year)
    #census_dict = clean_time_worked(census_dict, year)
    #census_dict = clean_generation_status(census_dict, year)
    #census_dict = clean_household_char(census_dict, year)
    #census_dict = clean_household_income(census_dict, year)
    #census_dict = clean_individual_income(census_dict, year)
    #census_dict = clean_industry(census_dict, year)
    #census_dict = clean_labour_force_status(census_dict, year)
    #census_dict = clean_commute_time(census_dict, year)
    #census_dict = clean_mobility(census_dict, year)
    #census_dict = clean_transport_mode(census_dict, year)
    census_dict = clean_occupation(census_dict, year)
    #census_dict = clean_workplace_status(census_dict, year)
    
        
    return census_dict

In [353]:
dict_2001 = clean_census(census_2001, 2001)
dict_2006 = clean_census(census_2006, 2006)
dict_2011 = clean_census(census_2011, 2011)
dict_2016 = clean_census(census_2016, 2016)

/Users/jq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
def create_subgroup_dict(df, year):
    
    # separate dataframe by 'Variables' containing regex expressions:
    if year == 2001:
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               '^Male', 
               '^Female', 
               'total - male', 
               'total - female']
        
    elif year == 2006:
        re1 = ['total.*by', 
               'population.*by',
               'common-law couples',
               '^Male[s\s,]', 
               '^Female[s\s,]',
              'total - mobility',
              'Average number of children']
        
    elif year == 2011:
        df.drop(index=201 , inplace=True)
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               'males',
               'Total population excluding institutional residents',  
               'Total.*in private households'] 
    elif year == 2016:
        re1 = ['^total', 'population.*by', 'males']
    
    subgroup = list(df[df.Variable.str.contains('|'.join(re1), flags=re.IGNORECASE)].index)
    subgroup.append(len(df.Variable)+1)
    subgroup = subgroup[1:]
    
    # create census dictionary of sub datasets
    # initialize variables for the lookup dictionary
    start = 0
    census_dict = {}
    
    for s in subgroup:
        sub_df = df.loc[start:s-1]

        # transpose dataframe and rename column
        sub_df = sub_df.set_index('Variable').T.reset_index().rename(columns={'index': 'LocalArea'})

        # check for duplicates and store dataframes into the dictionary
        if df.Variable[start] in census_dict:
            start = s
        else:
            census_dict[df.Variable[start]] = sub_df
            start = s
        
    return census_dict


In [7]:
def clean_age(census_dict, year):
    
    if year == 2001:
        column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                   '5 to 9 years', '10 to 14 years', '15 to 19 years',
                   '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                   '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                   '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                   '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                   '80 to 84 years', '85 to 89 years', '90 to 94 years',
                   '95 to 99 years', '100 years and over']

        male = census_dict['Male']
        female = census_dict['Female']
    
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
    
    else:
        if year == 2006:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                        '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 to 89 years', '90 to 94 years',
                       '95 to 99 years', '100 years and over', 'Median Age']
            
            total = census_dict['Male & Female, Total']
            male = census_dict['Male, Total']
            female = census_dict['Female, Total']
        
        elif year == 2011:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '15 years', '16 years', '17 years', '18 years', '19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over', 'Median age',
                       '% of the population aged 15 and over']
        
            total = census_dict['Total population by age groups']
            male = census_dict['Males, total']
            female = census_dict['Females, total']

        elif year == 2016:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 14 years', 
                       '0 to 4 years', '5 to 9 years', '10 to 14 years',
                       '15 to 64 years', '15 to 19 years', '20 to 24 years', 
                       '25 to 29 years', '30 to 34 years', '35 to 39 years', 
                       '40 to 44 years', '45 to 49 years', '50 to 54 years', 
                       '55 to 59 years', '60 to 64 years', '65 years and over', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over','85 to 89 years', 
                       '90 to 94 years', '95 to 99 years', '100 years and over']
        
            total = census_dict['Total - Age groups and average age of the population - 100% data']
            male = census_dict['Total - Age groups and average age of males - 100% data']
            female = census_dict['Total - Age groups and average age of females - 100% data']
         
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
    
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['population by age and sex'] = merged
    merged.to_csv('../../data/processed/census_' + str(year) + '/population_age_sex.csv')
    
    return census_dict

In [8]:
def clean_marital_status(census_dict, year):
    
    if year in [2001, 2006]:
        column_names = ['LocalArea', 'Total population 15 years and over',
                        'Single (never legally married)',
                       'Married', 'Separated', 'Divorced',
                       'Widowed', 'total x', 'Not living common law',
                       'Living common law']
        
        columns_ordered = ['LocalArea', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        df1 = census_dict['Total population 15 years and over by legal marital status']
        df2 = census_dict['Total population 15 years and over by common-law status']
        
        merged = pd.merge(df1, df2, on=['LocalArea'])
        merged.set_axis(column_names, axis=1, inplace=True)

        merged['Married or living with a or common-law partner'] = merged['Married'] + merged['Living common law']
        merged['Not living with a married spouse or common-law partner'] = merged['Total population 15 years and over'] - merged['Married or living with a or common-law partner']
        merged = merged[columns_ordered]
        
    
    else:
        if year == 2011:
            total = census_dict['Total population 15 years and over by marital status']
            male = census_dict['Males 15 years and over by marital status']
            female = census_dict['Females 15 years and over by marital status']
        elif year == 2016:
            total = census_dict['Total - Marital status for the population aged 15 years and over - 100% data']
            male = census_dict['Total - Marital status for males aged 15 years and over - 100% data']
            female = census_dict['Total - Marital status for females aged 15 years and over - 100% data']
            
        column_names = ['LocalArea', 'Type', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        
    census_dict['marital status'] = merged   
    merged.to_csv('../../data/processed/census_' + str(year) + '/marital_status.csv')
    return census_dict

In [9]:
def clean_private_households(census_dict, year):
    if year == 2011:
        column_names = ['LocalArea', 'Type', 'Persons in private households',
                   'Persons not in census families', 'Living with relatives',
                   'Living with non-relatives only', 'Living alone',
                   'Number of census family persons']
        
        total = census_dict['Total number of persons in private households']
        male = census_dict['Number of males in private households']
        female = census_dict['Number of females in private households']
        total_65 = census_dict['Total number of persons aged 65 years and over in private households']
        male_65 = census_dict['Number of males aged 65 years and over in private households']
        female_65 = census_dict['Number of females aged 65 years and over in private households']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
        
        female_65.insert(1, 'Type', '65+ female')
        female_65.set_axis(column_names, axis=1, inplace=True)
        male_65.insert(1, 'Type', '65+ male')
        male_65.set_axis(column_names, axis=1, inplace=True)
        total_65.insert(1, 'Type', '65+ total')
        total_65.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total, female_65, male_65, total_65])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['private households - individuals'] = merged
        merged.to_csv('../../data/processed/census_' + str(year) + '/private_households.csv')

    return census_dict

In [10]:
def clean_couple_fam_structure(census_dict, year):
    
    column_names = ['LocalArea', 'Type', 'Total', 
                    'Without children at home',
                    'With children at home', '1 child', '2 children',
                    '3 or more children']

    if year == 2016:
        total = census_dict['Total - Couple census families in private households - 100% data']
        total.insert(1, 'Type', 'total couples')
        total.set_axis(column_names, axis=1, inplace=True)
    
        census_dict['couples - family structure'] = total
        total.to_csv('../../data/processed/census_' + str(year) + '/couples_family_structure.csv')
        
    else:
        if year in [2011, 2006]:
            married = census_dict['Total couple families by family structure and number of children'] 
            married = married[['LocalArea', 'Married couples',
                               'Without children at home',
                               'With children at home', '1 child', '2 children',
                               '3 or more children']]
            common_law = census_dict['Common-law couples']
            
        elif year == 2001:
            married = census_dict['Total couple families by family structure']
            married = married[['LocalArea','Married couples', 
                               'Without children at home', 'With children at home',
                               '1 child', '2 children', '3 or more children']]
            common_law = census_dict['Common-law couples']
            
        married.insert(1, 'Type', 'married couples')
        married.set_axis(column_names, axis=1, inplace=True)
        common_law.insert(1, 'Type', 'common-law couples')
        common_law.set_axis(column_names, axis=1, inplace=True)
        
        merged = pd.concat([married, common_law])
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total couples'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
        
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['couples - family structure'] = merged
        merged.to_csv('../../data/processed/census_' + str(year) + '/couples_family_structure.csv')
    
    return census_dict
        

In [11]:
def clean_language_detailed(census_dict, year):
    
    if year == 2006:
        mt_total = census_dict['Total population by mother tongue']
        home_total = census_dict['Total population by language spoken most often at home']
        home_total = home_total.iloc[:,0:104].copy()
        work_total = census_dict['Total population 15 years and over who worked since January 1, 2005 by language used most often at work']
        
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        home_total.rename(columns={home_total.columns[1]:'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        work_total.rename(columns={work_total.columns[1]:'Total'}, inplace=True)
        work_total.insert(1, 'Type', 'language most often spoken at work - total')
        
        merged = pd.concat([mt_total, home_total, work_total])
    
    elif year == 2001:
        mt_total = census_dict['Total population by mother tongue']
        home_total = census_dict['Total population by home language']
        home_total = home_total.groupby(home_total.columns, axis=1).sum()
        
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        home_total.rename(columns={'Total population by home language':'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        
        merged = pd.concat([mt_total, home_total])
    
    else:
        if year == 2011:
            mt_total = census_dict['Detailed mother tongue - Total population excluding institutional residents']
            mt_male = census_dict['Detailed mother tongue - Males excluding institutional residents']
            mt_female = census_dict['Detailed mother tongue - Females excluding institutional residents']
        
            home_total = census_dict['Detailed language spoken most often at home - Total population excluding institutional residents']
            home_male = census_dict['Detailed language spoken most often at home - Males excluding institutional residents']
            home_female = census_dict['Detailed language spoken most often at home - Females excluding institutional residents']
        
            home2_total = census_dict['Detailed other language spoken regularly at home - Total population excluding institutional residents']
            home2_male = census_dict['Detailed other language spoken regularly at home - Males excluding institutional residents']
            home2_female = census_dict['Detailed other language spoken regularly at home - Females excluding institutional residents']
        
        elif year == 2016:
            mt_total = census_dict['Total - Mother tongue for the total population excluding institutional residents - 100% data']
            mt_male = census_dict['Total - Mother tongue for males excluding institutional residents - 100% data']
            mt_female = census_dict['Total - Mother tongue for females excluding institutional residents - 100% data']
        
            home_total = census_dict['Total - Language spoken most often at home for the total population excluding institutional residents - 100% data']
            home_male = census_dict['Total - Language spoken most often at home for males excluding institutional residents - 100% data']
            home_female = census_dict['Total - Language spoken most often at home for females excluding institutional residents - 100% data']
        
            home2_total = census_dict['Total - Other language(s) spoken regularly at home for the total population excluding institutional residents - 100% data']
            home2_male = census_dict['Total - Other language(s) spoken regularly at home for males excluding institutional residents - 100% data']
            home2_female = census_dict['Total - Other language(s) spoken regularly at home for females excluding institutional residents - 100% data']
        
        mt_female.rename(columns={mt_female.columns[1]:'Total'}, inplace=True)
        mt_female.insert(1, 'Type', 'mother tongue - female')
        mt_male.rename(columns={mt_male.columns[1]:'Total'}, inplace=True)
        mt_male.insert(1, 'Type', 'mother tongue - male')
        mt_total.rename(columns={mt_total.columns[1]:'Total'}, inplace=True)
        mt_total.insert(1, 'Type', 'mother tongue - total')
        
        home_female.rename(columns={home_female.columns[1]:'Total'}, inplace=True)
        home_female.insert(1, 'Type', 'language most often spoken at home - female')
        home_male.rename(columns={home_male.columns[1]:'Total'}, inplace=True)
        home_male.insert(1, 'Type', 'language most often spoken at home - male')
        home_total.rename(columns={home_total.columns[1]:'Total'}, inplace=True)
        home_total.insert(1, 'Type', 'language most often spoken at home - total')
        
        home2_female.rename(columns={home2_female.columns[1]:'Total'}, inplace=True)
        home2_female.insert(1, 'Type', 'other language spoken at home - female')
        home2_male.rename(columns={home2_male.columns[1]:'Total'}, inplace=True)
        home2_male.insert(1, 'Type', 'other language spoken at home - male')
        home2_total.rename(columns={home2_total.columns[1]:'Total'}, inplace=True)
        home2_total.insert(1, 'Type', 'other language spoken at home - total')
        
        merged = pd.concat([mt_female, mt_male, mt_total, home_female, home_male, home_total, home2_female, home2_male, home2_total])
    
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['detailed language'] = merged
    merged.to_csv('../../data/processed/census_' + str(year) + '/detailed_language.csv')
        
    return census_dict
    

In [12]:
def clean_official_language(census_dict, year):
    column_names = ['LocalArea', 'Type', 'Total',
                     'English', 'French', 'English and French',
                     'Neither English nor French']
    if year == 2016:
        known = census_dict['Total - Knowledge of official languages for the total population excluding institutional residents - 100% data']
        first = census_dict['Total - First official language spoken for the total population excluding institutional residents - 100% data']
    
    elif year == 2011:
        known = census_dict['Knowledge of official languages - Total population excluding institutional residents']
        first = census_dict['First official language spoken - Total population excluding institutional residents']
    
    elif year in [2001, 2006]:
        known = census_dict['Total population by knowledge of official languages']
        first = census_dict['Total population by first official language spoken']
            
        
    known.insert(1, 'Type', 'knowledge of official languages')
    known.set_axis(column_names, axis=1, inplace=True)
    first.insert(1, 'Type', 'first official language spoken')
    first.set_axis(column_names, axis=1, inplace=True)  
    
    merged = pd.concat([known, first])
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['official language'] = merged
    merged.to_csv('../../data/processed/census_' + str(year) + '/official_language.csv')
    
    return census_dict

In [13]:
def clean_structural_dwelling_type(census_dict, year):
          
    if year == 2006:
        column_names = ['LocalArea', 'Total',
       'Single-detached house', 'Semi-detached house', 'Row house',
       'Apartment, duplex',
       'Apartment, building that has five or more storeys']
        
        df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
    
    elif year in [2001, 2011, 2016]:
        column_names = ['LocalArea', 'Total',
       'Single-detached house', 'Semi-detached house', 'Row house',
       'Apartment, detached duplex',
       'Apartment, building that has five or more storeys',
       'Apartment, building that has fewer than five storeys',
       'Other single-attached house', 'Movable dwelling']
        
        if year == 2001:
            df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
            df = df.iloc[:,0:10].copy()
        
        elif year == 2011:
            df = census_dict['Total number of occupied private dwellings by structural type of dwelling']
            df = df[['LocalArea',
                    'Total number of occupied private dwellings by structural type of dwelling',
                    'Single-detached house', 'Semi-detached house', 'Row house',
                    'Apartment, duplex', 
                    'Apartment, building that has five or more storeys', 
                    'Apartment, building that has fewer than five storeys',
                    'Other single-attached house', 'Movable dwelling']].copy()
    
        elif year == 2016:
            df = census_dict['Total - Occupied private dwellings by structural type of dwelling - 100% data']
            df = df[['LocalArea',
            'Total - Occupied private dwellings by structural type of dwelling - 100% data',
           'Single-detached house', 'Semi-detached house', 'Row house',
           'Apartment or flat in a duplex',
           'Apartment in a building that has five or more storeys',
           'Apartment in a building that has fewer than five storeys',
           'Other single-attached house', 'Movable dwelling']].copy()
        
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['structual dwelling type'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/structual_dwelling_type.csv')
    
    return census_dict

In [14]:
def clean_household_size(census_dict, year):
    if year == 2001:
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 to 5 persons',
       '6 or more persons',
       'Average household size']
    
        df = census_dict['Total number of private households by household size']
        
    elif year in [2006, 2011]:
        
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 to 5 persons',
       '6 or more persons', 'Number of persons in private households',
       'Average household size']
    
        df = census_dict['Total number of private households by household size']
        
    elif year == 2016:
        
        column_names = ['LocalArea', 'Total households',
       '1 person', '2 persons', '3 persons', '4 persons', '5 or more persons',
       'Number of persons in private households', 'Average household size']
        
        df = census_dict['Total - Private households by household size - 100% data']
   
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['household size'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/household_size.csv')
    
    return census_dict

In [15]:
def clean_lone_parent(census_dict, year):
    column_names = ['LocalArea', 'Total lone-parent families',
                 'Female parent', 'Male parent',
                 '1 child', '2 children', '3 or more children']
    if year == 2016:
        df1 = census_dict["Total lone-parent families by sex of parent"]
        df2 = census_dict["Total - Lone-parent census families in private households - 100% data"]
        
        df = pd.concat([df1, df2], axis=1)
        df = df.groupby(df.columns, axis=1).first()
        df = df[['LocalArea', 'Total lone-parent families by sex of parent',
                 'Female parent', 'Male parent',
                 '1 child', '2 children', '3 or more children']].copy()
    
    elif year == 2011:
        df = census_dict['Total lone-parent families by sex of parent and number of children']
        df = df.groupby(df.columns, axis=1).sum()
        df = df[['LocalArea', 'Total lone-parent families by sex of parent and number of children',
                  'Female parent', 'Male parent',
                 '1 child', '2 children', '3 or more children']].copy()
    
    elif year == 2006:
        df1 = census_dict['Total lone-parent families by sex of parent and number of children']
        df2 = census_dict['Female parent']
        df2 = df2.iloc[:,1:5].copy()
        df3 = census_dict['Male parent']
        df3 = df3.iloc[:,1:5].copy()
        
        df = pd.concat([df1, df2, df3], axis=1)
        df = df.groupby(df.columns, axis=1).sum()
        df = df[['LocalArea', 'Total lone-parent families by sex of parent and number of children',
                  'Female parent', 'Male parent',
                 '1 child', '2 children', '3 or more children']].copy()
    
    elif year == 2001:
        df1 = census_dict['Total lone-parent families by sex of parent']
        df2 = census_dict['Female parent']
        df2 = df2.iloc[:,1:5].copy()
        df3 = census_dict['Male parent']
        df3 = df3.iloc[:,1:5].copy()
        
        df = pd.concat([df1, df2, df3], axis=1)
        df = df.groupby(df.columns, axis=1).sum()
        df = df[['LocalArea', 'Total lone-parent families by sex of parent',
                  'Female parent', 'Male parent',
                 '1 child', '2 children', '3 or more children']].copy()

    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['lone_parent'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/lone_parent.csv')
        
    return census_dict

In [16]:
def clean_immigration_age(census_dict, year):
    
    if year in [2006, 2016]:
        column_names = ['LocalArea', 
                 'Total immigrant population',
                 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                 '25 to 44 years', '45 years and over']
        
        if year == 2006:
            df = census_dict['Total immigrant population by age at immigration']
            
        elif year == 2016:
            df = census_dict['Total - Age at immigration for the immigrant population in private households - 25% sample data']
    
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 
                 'Total immigrant population',
                 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                 '25 to 44 years', '45 years and over']
        
        df = pd.read_csv('../../data/processed/nhs/Age at immigration.csv', index_col=0)
        df = df[['LocalArea', 'Type', 
                 'Total immigrant population in private households by age at immigration',
                 'Under 5 years', '5 to 14 years', '15 to 24 years', 
                 '25 to 44 years', '45 years and over']].copy()
    
    elif year == 2001:
        column_names = ['LocalArea', 
                 'Total immigrant population',
                 'Under 5 years', '5 to 19 years', '20 years and over']
        df = census_dict['Total immigrant population by age at immigration']
       
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['immigration_age'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/immigration_age.csv')
            
    return census_dict

In [17]:
def clean_immigration_period(census_dict, year):
    
    if year == 2001:
        column_names = ['LocalArea', 
                 'Total immigrant population','Before 1961', 
                        '1961 to 1970', '1971 to 1980', '1981 to 1990', 
                        '1991 to 1995', '1996 to 2001']
        df = census_dict['Total immigrant population by period of immigration']
    
    elif year == 2006:
        column_names = ['LocalArea', 
                 'Total immigrant population',
                 'Before 1961', '1961 to 1970', '1971 to 1980', '1981 to 1990',
                 '1991 to 2000', '1991 to 1995', '1996 to 2000', '2001 to 2006']
        df = census_dict['Total immigrant population by period of immigration']
            
    elif year == 2016:
        column_names = ['LocalArea', 'Total population',
           'Non-immigrants', 'Non-permanent residents', 'Immigrants', 
           'Before 1981', '1981 to 1990', '1991 to 2000', '2001 to 2010', 
           '2001 to 2005', '2006 to 2010','2011 to 2016']
                        
        df = census_dict['Total - Immigrant status and period of immigration for the population in private households - 25% sample data']
        df = df[['LocalArea',
           'Total - Immigrant status and period of immigration for the population in private households - 25% sample data',
           'Non-immigrants', 'Non-permanent residents', 'Immigrants', 'Before 1981', '1981 to 1990',
           '1991 to 2000', '2001 to 2010', '2001 to 2005', '2006 to 2010',
           '2011 to 2016']].copy()
    
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population',
                  'Non-immigrants', 'Non-permanent residents', 'Immigrants',
                 'Before 1971', '1971 to 1980', '1981 to 1990', '1991 to 2000', '2001 to 2005']
        
        df = pd.read_csv('../../data/processed/nhs/Immigrant status and period of immigration.csv', index_col=0)
        df = df[['LocalArea', 'Type', 
                 'Total population in private households by immigrant status and period of immigration',
                  'Non-immigrants', 'Non-permanent residents', 'Immigrants',
                 'Before 1971', '1971 to 1980', '1981 to 1990', '1991 to 2000', '2001 to 2005']].copy()
    
       
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['immigration_period'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/immigration_period.csv')
    
    return census_dict

In [18]:
def clean_visible_minority(census_dict, year):
    
    column_names = ['LocalArea', 'Total population',
                    'Not a visible minority', 'Total visible minority population',
                    'Arab', 'Black', 'Chinese', 'Filipino', 'Japanese', 'Korean',
                    'Latin American', 'West Asian', 'South Asian', 'Southeast Asian', 
                    'Multiple visible minorities', 'Other visible minority']
    
    if year == 2001:
        
        df = census_dict['Total population by visible minority groups']
        df = df[['LocalArea', 'Total population by visible minority groups',
                 'All others', 'Total visible minority population', 
                 'Arab', 'Black', 'Chinese',  'Filipino', 'Japanese',
                 'Korean', 'Latin American', 'West Asian', 'South Asian',
                 'Southeast Asian', 'Multiple visible minorities',
                 'Visible minority, n.i.e.']].copy()
        
    elif year == 2006:
        
        df = census_dict['Total population by visible minority groups']
        df = df[['LocalArea', 'Total population by visible minority groups',
                 'Not a visible minority', 'Total visible minority population',
                 'Arab', 'Black', 'Chinese', 'Filipino', 'Japanese', 'Korean',
                 'Latin American', 'West Asian', 'South Asian', 'Southeast Asian', 
                 'Multiple visible minority', 'Visible minority, n.i.e.']]
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population',
                       'Not a visible minority', 'Total visible minority population',
                       'Arab', 'Black', 'Chinese', 'Filipino', 'Japanese', 'Korean',
                       'Latin American', 'West Asian', 'South Asian', 'Southeast Asian', 
                       'Multiple visible minorities', 'Other visible minority']
        
        df = pd.read_csv('../../data/processed/nhs/Visible minority population.csv', index_col=0)
        df = df[['LocalArea', 'Type', 
                 'Total population in private households by visible minority',
                 'Not a visible minority', 'Total visible minority population',
                 'Arab', 'Black', 'Chinese', 'Filipino', 'Japanese', 'Korean',
                 'Latin American', 'West Asian', 'South Asian', 'Southeast Asian', 
                 'Multiple visible minorities', 'Visible minority, n.i.e.']].copy()
        
    elif year == 2016:
        df = census_dict['Total visible minority population']
        df = df[['LocalArea', 'Total visible minority population',
                 'Not a visible minority', 'Total visible minority population',
                 'Arab', 'Black', 'Chinese', 'Filipino', 'Japanese', 'Korean',
                 'Latin American', 'West Asian', 'South Asian', 'Southeast Asian', 
                 'Multiple visible minorities', 'Visible minority, n.i.e.']]
       
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['visible_minority'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/visible_minority.csv')
    
    return census_dict

In [19]:
def clean_birth_place(census_dict, year):
    if year == 2001:
        column_names = ['LocalArea', 'Total population',
                'Non-immigrants', 
                'Born in province of residence',
                'Born outside province of residence', 
                'Non-permanent residents',
                'Immigrants', 
                'United Kingdom', "China", 
                'Italy', 'India', 'United States', 'Hong Kong', 
                'Philippines', 'Poland', 'Germany', 'Portugal', 'Viet Nam', 
                'Jamaica', 'Netherlands', 'Guyana', 'Greece', 'South Korea', 
                'France', 'Lebanon', 'Taiwan', 'Yugoslavia', 'Haiti', 'Ukraine', 
                'Croatia', 'Mexico', 'Egypt', 'South Africa', 
                'Ireland', 'Morocco', 'Austria', 'Switzerland', 
                'Other places of birth']
        
        df1 = census_dict['Total population by immigrant status and place of birth']
        df1 = df1.iloc[:, 1:5].copy()
        df2 = census_dict['Total immigrants by selected places of birth']
        
        df = pd.concat([df1, df2], axis=1)
        df = df[['LocalArea', 'Total population by immigrant status and place of birth',
                'Non-immigrant population', 
                'Born in province of residence',
                'Born outside province of residence', 
                'Non-permanent residents',
                'Total immigrants by selected places of birth', 
                'United Kingdom', "China, People's Republic of", 
                'Italy', 'India', 'United States', 'Hong Kong, Special Administrative Region', 
                'Philippines', 'Poland', 'Germany', 'Portugal', 'Viet Nam', 
                'Jamaica', 'Netherlands', 'Guyana', 'Greece', 'Korea, South', 
                'France', 'Lebanon', 'Taiwan', 'Yugoslavia', 'Haiti', 'Ukraine', 
                 'Croatia', 'Mexico', 'Egypt', 'South Africa, Republic of', 
                 'Ireland, Republic of (EIRE)', 'Morocco',
                'Austria', 'Switzerland', 'All other places of birth']].copy()
    
    elif year == 2006:
        column_names = ['LocalArea', 'Total population',
                'Non-immigrants', 
                'Born in province of residence',
                'Born outside province of residence', 
                'Non-permanent residents', 'Immigrants',
                'United States', 'Central America', 'Caribbean and Bermuda',
                'South America', 'Europe', 'Western Europe', 'Eastern Europe',
                'Southern Europe', 'Italy', 'Other Southern Europe', 'Northern Europe',
                'United Kingdom', 'Other Northern Europe', 'Africa', 'Western Africa',
                'Eastern Africa', 'Northern Africa', 'Central Africa',
                'Southern Africa', 'Asia and the Middle East',
                'West Central Asia and the Middle East', 'Eastern Asia',
                'China', 'Hong Kong', 'Other Eastern Asia',
                'Southeast Asia', 'Philippines', 'Other Southeast Asia',
                'Southern Asia', 'India', 'Other Southern Asia', 'Oceania and other']
        
        df = census_dict['Total population by immigrant status and place of birth']
        df = df[['LocalArea', 'Total population by immigrant status and place of birth',
                'Non-immigrants', 
                'Born in province of residence',
                'Born outside province of residence', 
                'Non-permanent residents', 'Immigrants',
                'United States of America', 'Central America', 'Caribbean and Bermuda',
                'South America', 'Europe', 'Western Europe', 'Eastern Europe',
                'Southern Europe', 'Italy', 'Other Southern Europe', 'Northern Europe',
                'United Kingdom', 'Other Northern Europe', 'Africa', 'Western Africa',
                'Eastern Africa', 'Northern Africa', 'Central Africa',
                'Southern Africa', 'Asia and the Middle East',
                'West Central Asia and the Middle East', 'Eastern Asia',
                "China, People's Republic of",
                'Hong Kong, Special Administrative Region', 'Other Eastern Asia',
                'Southeast Asia', 'Philippines', 'Other Southeast Asia',
                'Southern Asia', 'India', 'Other Southern Asia', 'Oceania and other']]
            
    
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population',
                 'Non-immigrants',
                 'Born in province of residence',
                 'Born outside province of residence',
                 'Non-permanent residents', 'Immigrants',
                 'Afghanistan', 'Africa', 'Algeria', 'Americas', 'Asia',
                 'Bangladesh', 'Bosnia and Herzegovina', 'Chile',
                 'China', 'Colombia', 'Croatia', 'Egypt', 'El Salvador', 'Ethiopia',
                 'Europe', 'Fiji', 'France', 'Germany', 'Greece', 'Guyana', 'Haiti',
                 'Hong Kong', 'Hungary', 'India', 'Iran', 'Iraq', 'Ireland', 'Italy', 'Jamaica',
                 'Japan', 'Kenya', 'South Korea', 'Lebanon', 'Mexico', 'Morocco',
                 'Netherlands', 'Nigeria', 'Pakistan', 'Peru', 'Philippines', 'Poland',
                 'Portugal', 'Romania', 'Russia', 'Serbia', 'South Africa',
                 'Sri Lanka', 'Taiwan', 'Trinidad and Tobago', 'Turkey', 'Ukraine', 'United Kingdom',
                'United States', 'Viet Nam', 'Oceania and other',
                'Other Africa', 'Other Americas',
                'Other Asia', 'Other Europe', 
                'Other places of birth']
        
        df = pd.read_csv('../../data/processed/nhs/Immigrant status and selected places of birth.csv', index_col=0)
        df = df[['LocalArea', 'Type',  
                 'Total population in private households by immigrant status and selected places of birth',
                 'Non-immigrants',
                 'Born in province of residence',
                 'Born outside province of residence',
                 'Non-permanent residents', 'Immigrants',
                 'Afghanistan', 'Africa', 'Algeria', 'Americas', 'Asia',
                 'Bangladesh', 'Bosnia and Herzegovina', 'Chile',
                 'China', 'Colombia', 'Croatia', 'Egypt', 'El Salvador', 'Ethiopia',
                 'Europe', 'Fiji', 'France', 'Germany', 'Greece', 'Guyana', 'Haiti',
                 'Hong Kong Special Administrative Region', 'Hungary',
                 'India', 'Iran', 'Iraq', 'Ireland, Republic of', 'Italy', 'Jamaica',
                 'Japan', 'Kenya', 'Korea, South', 'Lebanon', 'Mexico', 'Morocco',
                 'Netherlands', 'Nigeria', 'Pakistan', 'Peru', 'Philippines', 'Poland',
                 'Portugal', 'Romania', 'Russian Federation', 'Serbia', 'South Africa, Republic of',
                 'Sri Lanka', 'Taiwan', 'Trinidad and Tobago', 'Turkey', 'Ukraine', 'United Kingdom',
                'United States', 'Viet Nam', 'Oceania and other',
                'Other places of birth in Africa', 'Other places of birth in Americas',
                'Other places of birth in Asia', 'Other places of birth in Europe', 
                 'Other places of birth']].copy()
    
    elif year == 2016:
        column_names = ['LocalArea', 'Total population',
           'Non-immigrants', 'Non-permanent residents', 'Immigrants',
            'Americas', 'Brazil', 'Colombia', 'El Salvador', 'Guyana', 'Haiti',
            'Jamaica', 'Mexico', 'Peru', 'Trinidad and Tobago', 'United States',
               'Other  Americas', 'Europe', 'Bosnia and Herzegovina',
               'Croatia', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
               'Netherlands', 'Poland', 'Portugal', 'Romania', 'Russia',
               'Serbia', 'Ukraine', 'United Kingdom',
               'Other Europe', 'Africa', 'Algeria', 'Egypt',
               'Ethiopia', 'Kenya', 'Morocco', 'Nigeria', 'Somalia',
               'South Africa', 'Other Africa', 'Asia',
               'Afghanistan', 'Bangladesh', 'China', 'Hong Kong', 'India', 'Iran',
               'Iraq', 'Japan', 'South Korea', 'Lebanon', 'Pakistan', 'Philippines',
               'Sri Lanka', 'Syria', 'Taiwan', 'Viet Nam',
               'Other Asia', 'Oceania and other places of birth']
        
        df1 = census_dict['Total - Immigrant status and period of immigration for the population in private households - 25% sample data']
        df1 = df1[['Total - Immigrant status and period of immigration for the population in private households - 25% sample data', 'Non-immigrants', 'Non-permanent residents']].copy()
        df2 = census_dict['Total - Selected places of birth for the immigrant population in private households - 25% sample data']

        df = pd.concat([df1, df2], axis=1)
        df = df[['LocalArea', 
                 'Total - Immigrant status and period of immigration for the population in private households - 25% sample data',
                'Non-immigrants', 'Non-permanent residents', 
                'Total - Selected places of birth for the immigrant population in private households - 25% sample data',
                'Americas', 'Brazil', 'Colombia', 'El Salvador', 'Guyana', 'Haiti',
                'Jamaica', 'Mexico', 'Peru', 'Trinidad and Tobago', 'United States',
               'Other places of birth in Americas', 'Europe', 'Bosnia and Herzegovina',
               'Croatia', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
               'Netherlands', 'Poland', 'Portugal', 'Romania', 'Russian Federation',
               'Serbia', 'Ukraine', 'United Kingdom',
               'Other places of birth in Europe', 'Africa', 'Algeria', 'Egypt',
               'Ethiopia', 'Kenya', 'Morocco', 'Nigeria', 'Somalia',
               'South Africa, Republic of', 'Other places of birth in Africa', 'Asia',
               'Afghanistan', 'Bangladesh', 'China', 'Hong Kong', 'India', 'Iran',
               'Iraq', 'Japan', 'Korea, South', 'Lebanon', 'Pakistan', 'Philippines',
               'Sri Lanka', 'Syria', 'Taiwan', 'Viet Nam',
               'Other places of birth in Asia', 'Oceania and other places of birth']].copy()
       
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['immigration_birth_place'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/immigration_birth_place.csv')
    
    return census_dict

In [20]:
def clean_shelter_tenure(census_dict, year):
    
    column_names = ['LocalArea', 'Total number of dwellings', 'Owned',
                       'Rented', 'Band housing']

    if year == 2001:
        df = census_dict['Total number of occupied private dwellings by tenure']
        
    elif year == 2006:
        df = census_dict['Total number of occupied private dwellings by housing tenure']
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total number of dwellings', 
                        'Owned','Rented']
        
        df = pd.read_csv('../../data/processed/nhs/Shelter costs.csv', index_col=0)
        df = df[['LocalArea','Type',
                 'Total number of owner and tenant households with household total income greater than zero, in non-farm, non-reserve private dwellings by shelter-cost-to-income ratio',
                 'Number of owner households in non-farm, non-reserve private dwellings',
                 'Number of tenant households in non-farm, non-reserve private dwellings']].copy()
        
    elif year == 2016:
        df = census_dict['Total - Private households by tenure - 25% sample data']
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['shelter_tenure'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/shelter_tenure.csv')
    
    return census_dict

# Incomplete Helper Functions

In [14]:
def clean_household_type(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['household_type'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/household_type.csv')
    
    return census_dict

IndentationError: expected an indented block (<ipython-input-14-c1d5e90acc76>, line 5)

In [328]:
def clean_census_family_children(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['census_family_children'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/census_family_children.csv')
    
    return census_dict

In [330]:
def clean_aboriginal(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Aboriginal population.csv', index_col=0)
        
    elif year == 2016:
        
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['aboriginal'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/aboriginal.csv')
    
    return census_dict

In [332]:
def clean_citizenship(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Citizenship.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['citizenship'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/citizenship.csv')
    
    return census_dict

In [333]:
def clean_worker_class(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Class of worker.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['worker_class'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/worker_class.csv')
    
    return census_dict

In [334]:
def clean_education(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Education.csv', index_col=0)
        df = df[['LocalArea', 'Type', 
                 'Total population aged 15 years and over by highest certificate, diploma or degree',
                 'Total population aged 25 to 64 years by highest certificate, diploma or degree',
                 'Apprenticeship or trades certificate or diploma',
                 'College, CEGEP or other non-university certificate or diploma',
                 'University certificate or diploma below bachelor level',
                 'University certificate, diploma or degree above bachelor level',
                 'University certificate, diploma or degree at bachelor level or above',
                 "Bachelor's degree",
                 'High school diploma or equivalent',
                 'No certificate, diploma or degree',
                 'No postsecondary certificate, diploma or degree',
                 'Postsecondary certificate, diploma or degree',
                 'With postsecondary certificate, diploma or degree',
                 'Total population aged 15 years and over by location of study compared with province or territory of residence',
                  'Location of study inside Canada', 'Location of study outside Canada',
                  'Same as province or territory of residence',
                 'Another province or territory',
                 'Total population aged 15 years and over by major field of study - Classification of Instructional Programs (CIP) 2011',
                  'Agriculture, natural resources and conservation',
                  'Architecture, engineering, and related technologies',
                 'Business, management and public administration',
                 'Education', 'Health and related fields', 'Humanities',
                 'Mathematics, computer and information sciences',
                 'Personal, protective and transportation services',
                 'Physical and life sciences and technologies',
                 'Social and behavioural sciences and law',
                 'Visual and performing arts, and communications technologies',
                 'Other fields of study']]
        
    elif year == 2016:
        
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['education'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/education.csv')
    
    return census_dict

In [335]:
def clean_ethnic_origin(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Ethnic origin population.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['ethnic_origin'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/ethnic_origin.csv')
    
    return census_dict

In [288]:
def clean_time_worked(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['time_worked'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/time_worked.csv')
    
    return census_dict

In [289]:
def clean_generation_status(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Generation status.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['generation_status'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/generation_status.csv')
    
    return census_dict

In [290]:
def clean_household_char(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
         column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Household characteristics.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['household_characteristics'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/household_characteristics.csv')
    
    return census_dict

In [339]:
def clean_household_income(census_dict, year):
    if year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Income of households in 2010.csv', index_col=0)
        df = df[['LocalArea', 'Type', 
                 'After-tax income of households in 2010 of private households',
                 '$5,000 to $9,999', '$10,000 to $14,999', '$15,000 to $19,999',  
                 '$20,000 to $29,999', '$30,000 to $39,999', '$40,000 to $49,999', 
                 '$50,000 to $59,999', '$60,000 to $79,999', '$80,000 to $99,999',
                 '$100,000 and over', '$100,000 to $124,999', '$125,000 and over',
                 '$125,000 to $149,999', '$150,000 and over',
       'Average after-tax household income ($)',
       'Average household total income ($)',
       'Average household total income ($)',
       'Household income in 2010 of private households',
       'Household total income in 2010 of private households',
       'Median after-tax household income ($)',
       'Median household total income ($)',
       'Median household total income ($)', 'One-person private households',
       'Two-or-more-persons private households', 'Under $5,000', 'LocalArea']]
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['household_income'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/household_income.csv')
    
    return census_dict

In [340]:
def clean_individual_income(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Income of individuals in 2010.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['individual_income'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/individual_income.csv')
    
    return census_dict

In [341]:
def clean_industry(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Industry.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['industry'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/industry.csv')
    
    return census_dict

In [342]:
def clean_labour_force_status(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Labour force status.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['labour_force_status'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/labour_force_status.csv')
    
    return census_dict

In [343]:
def clean_commute_time(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Median commuting duration.csv', index_col=0)
        
    elif year == 2016:
        
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['commute_time'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/commute_time.csv')
    
    return census_dict

In [344]:
def clean_mobility(census_dict, year):
    
    if year == 2001:
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Mobility.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['mobility'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/mobility.csv')
    
    return census_dict

In [376]:
def clean_transport_mode(census_dict, year):

    column_names = ['LocalArea', 'Type', 'Total population aged 15', 'Bicycle',
                    'Car as a driver', 'Car as a passenger',
                    'Other methods', 'Public transit', 'Walked']

    if year == 2001:
        return True

    elif year == 2006:
        return True

    elif year == 2011:
        df = pd.read_csv('../../data/processed/nhs/Mode of transportation.csv',
                         index_col=0).iloc[:, [-1, 0, -3, 1, 2, 3, 4, 5, -2]]
        df.columns = column_names

    elif year == 2016:
        return True

    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    #census_dict['transport_mode'] = df
    #df.to_csv('../../data/processed/census_' +
    #          str(year) + '/transport_mode.csv')

    return df


In [374]:
dict_2006['Males with usual place of work or no fixed workplace address'].head()

Variable,LocalArea,Males with usual place of work or no fixed workplace address,"Car, truck, van, as driver","Car, truck, van, as passenger",Public transit,Walked,Bicycle,Motorcycle,Taxicab,Other method
0,Arbutus-Ridge,2850,2165,120,335,95,110,10,0,15
1,Downtown,12925,5500,280,2050,4440,315,50,75,200
2,Dunbar-Southlands,4890,3555,220,615,165,260,30,0,40
3,Fairview,7735,4130,135,1510,1360,460,40,25,75
4,Grandview-Woodland,7420,3625,290,2190,575,580,115,15,30


In [377]:
clean_transport_mode(dict_2011, 2011)

,LocalArea,Type,Total population aged 15,Bicycle,Car as a driver,Car as a passenger,Other methods,Public transit,Walked
2,Arbutus-Ridge,Female,2715.0,25.0,1610.0,150.0,25.0,675.0,195.0
1,Arbutus-Ridge,Male,2745.0,45.0,2005.0,0.0,45.0,465.0,80.0
0,Arbutus-Ridge,Total,5465.0,100.0,3605.0,180.0,90.0,1140.0,310.0
0,Downtown,Total,31605.0,735.0,10615.0,575.0,465.0,7265.0,11965.0
1,Downtown,Male,17465.0,475.0,6560.0,75.0,325.0,3600.0,6315.0
2,Downtown,Female,14140.0,170.0,4055.0,430.0,85.0,3665.0,5665.0
0,Dunbar-Southlands,Total,9370.0,490.0,6175.0,395.0,145.0,1725.0,425.0
1,Dunbar-Southlands,Male,5040.0,295.0,3465.0,100.0,25.0,915.0,135.0
2,Dunbar-Southlands,Female,4340.0,165.0,2705.0,265.0,50.0,815.0,285.0
0,Fairview,Total,18510.0,1045.0,7540.0,525.0,320.0,5915.0,3190.0


In [348]:
def clean_occupation(census_dict, year):

    column_names = ['LocalArea', 'Type', 'Total', 'All occupations', 'Occupations n/a',
                    'Management', 'Business and finance', 'Natural and applied sciences',
                    'Health', 'Social Sience and education', 'Art', 'Sales and service',
                    'Trades and transport', 'Natural resources and agriculture', 'Manufacturing and utilities']

    if year == 2001 or year == 2006:
        if year == 2001:
            female = census_dict['Female labour force 15 years and over - Occupation']
            male = census_dict['Male labour force 15 years and over - Occupation']
        else:
            female = census_dict['Female labour force 15 years and over by occupation - National Occupational Classification for Statistics 2006']
            male = census_dict['Male labour force 15 years and over by occupation - National Occupational Classification for Statistics 2006']

        occupations = [i for i in female.columns if re.match(r'^[A-Z] ', i)]
        female['Type'] = ['Female'] * len(female)
        female = pd.concat(
            [female.iloc[:, [0, -1, 1, 3, 2]], female.loc[:, occupations]], axis=1)
        female.columns = column_names

        male['Type'] = ['Male'] * len(male)
        male = pd.concat(
            [male.iloc[:, [0, -1, 1, 3, 2]], male.loc[:, occupations]], axis=1)
        male.columns = column_names

        df = pd.concat([female, male])

        total = df.groupby(['LocalArea']).sum().reset_index()
        total['Type'] = ['Total'] * len(total)

        df = pd.concat([df, total])

    elif year == 2011:
        df = pd.read_csv(
            '../../data/processed/nhs/Occupation.csv', index_col=0)

        df = pd.concat(
            (df.iloc[:, [14, 0, 13, 11, 12]], df.iloc[:, 1:11]), axis=1)

    elif year == 2016:
        total = census_dict['Total labour force population aged 15 years and over by occupation - National Occupational Classification (NOC) 2016 - 25% sample data']
        female = census_dict['Total female labour force population aged 15 years and over by occupation - National Occupational Classification (NOC) 2016 - 25% sample data']
        male = census_dict['Total male labour force population aged 15 years and over by occupation - National Occupational Classification (NOC) 2016 - 25% sample data']
        
        male['Type'] = ['Male'] * len(male)
        total['Type'] = ['Total'] * len(total)
        female['Type'] = ['Female'] * len(female)
        
        total = pd.concat(
            (total.iloc[:, [0, -1, 1, 3, 2]], total.iloc[:, 4:14]), axis=1)
        female = pd.concat(
            (female.iloc[:, [0, -1, 1, 3, 2]], female.iloc[:, 4:14]), axis=1)
        male = pd.concat(
            (male.iloc[:, [0, -1, 1, 3, 2]], male.iloc[:, 4:14]), axis=1)
        
        female.columns, male.columns, total.columns = column_names, column_names, column_names
        
        df = pd.concat([total, female, male])

    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['occupation'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/occupation.csv')

    return census_dict


In [347]:
def clean_workplace_status(census_dict, year):
    
    columns = ['LocalArea']
    
    if year == 2001:
        df = 
        
    elif year == 2006:
        
    elif year == 2011:
        column_names = ['LocalArea', 'Type', 'Total population']
        
        df = pd.read_csv('../../data/processed/nhs/Place of work status.csv', index_col=0)
        
    elif year == 2016:
    
    df.set_axis(column_names, axis=1, inplace=True)
    df.sort_values(by=['LocalArea'], inplace=True)
    census_dict['workplace_status'] = df
    df.to_csv('../../data/processed/census_' + str(year) + '/workplace_status.csv')
    
    return census_dict